In [ ]:
import wave
from pyaudio import PyAudio,paInt16
import os
from aip import AipSpeech
import ffmpy #视频音频转码模块
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import sys
import pygame
from pydub import AudioSegment
import requests

os.chdir('E:\我的东西\新建文件夹\文件\课件\信工课件\电子系统综合设计\半成品\代码文件')
framerate=8000
NUM_SAMPLES=2000
channels=1
sampwidth=2
TIME=1
filename_record = 'audio_record.wav'
filename_answer = 'audio.wav'
APP_ID = '11169887'
API_KEY = 'TQypLIsDnr4XwzfyKGLqMsfD'
SECRET_KEY = 'bc5efee36b796c2b467dba12c2a080b0'


def save_wave_file(filename,data):
    '''save the date to the wavfile'''
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)
    wf.writeframes(b"".join(data))
    wf.close()

def my_record():
    pa=PyAudio()
    stream=pa.open(format = paInt16,channels=1,
                   rate=framerate,input=True,
                   frames_per_buffer=NUM_SAMPLES)
    my_buf=[]
    count=0
    print('Recording:',end='')
    while count<TIME*20:#控制录音时间
        string_audio_data = stream.read(NUM_SAMPLES)
        my_buf.append(string_audio_data)
        count+=1
        print('.',end='')
    print()
    stream.close()
    return my_buf

chunk=2048 #好像是分段记录时每次记录的点数
def play(filename): #可以播放mp3,wav,amr,mp4; pcm还不行，要额外指定sample_width
    chunk = 1024
    filename_tmp = 'tmp.wav'
    try:
        sound = AudioSegment.from_file(filename, format=str.split(filename,'.')[1])
    except:
        filename_answer = 'tmp.' + str.split(filename,'.')[1]
#         filename_answer = 'tmp.wav'
        if os.path.exists(filename_answer):
            os.remove(filename_answer)
        ff = ffmpy.FFmpeg( 
        inputs={filename: None},
        outputs={filename_answer: None}
        )
        ff.run() #美中不足的时没有学会怎么执行覆盖文件操作，文件存在时写入操作会被直接跳过，故现在只能暂时在转码播放后直接将其移除            
        sound = AudioSegment.from_file(filename_answer, format=str.split(filename_answer,'.')[1])        
    p=PyAudio()
    stream=p.open(format=p.get_format_from_width(sound.sample_width),channels= \
        sound.channels,rate=sound.frame_rate,output=True)
    i=0
    while True:
        data = sound.raw_data[i*chunk : (i+1)*chunk]
        if data==b'':break
        stream.write(data) #这一行是播放语音
        i = i + 1
    stream.close()
    p.terminate()       
    

# 读取文件
def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read() 
    
def get_weather(message):
    date_heard = re.search(u'.*今天'.encode('utf8'), message)
    if date_heard is not None:
        day = 0
    else:
        date_heard = re.search(u'.*明天'.encode('utf8'), message)
        if date_heard is not None:
            day = 1
        else: 
            date_heard = re.search(u'.*后天'.encode('utf8'), message)
            if date_heard is not None:
                day = 2
    resp=urlopen('http://www.weather.com.cn/weather/101280101.shtml')
    soup=BeautifulSoup(resp,'html.parser')
    content = soup.findAll('li', class_="sky") #返回一个链表，每个元素是一个标签包括一天的天气内容
    tagToday=content[day].find('p', class_="tem")
    try:
        temperatureHigh=tagToday.span.string
    except AttributeError as e:
        temperatureHigh=tagToday.find_next('p', class_="tem").span.string
    temperatureLow=tagToday.i.string
    weather=soup.find('p', class_="wea").string
    tagWind=soup.find('p',class_="win")
    winL=tagWind.i.string
    time_now = datetime.datetime.now()
    delta = datetime.timedelta(days=day)
    time_day = time_now + delta
    date_string = str(time_day.month) + '月' + str(time_day.day) + '日' 
    name_date = ['今', '明', '后']
    result_weather = name_date[day] + '天是：'+ date_string + ',风级：' + winL + ',最低温度：' + temperatureLow.replace('℃','度') + ',最高温度：' + temperatureHigh + '度' + ',天气：'+ weather
    return result_weather

def say_answer(message_answer): #client需要在外部创建一个公用的，因为每次向百度语音API申请一个太花费时间了
    result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 3}) 
    if not isinstance(result_tts, dict):
        with open('audio_raw.wav', 'wb') as f:
            f.write(result_tts)
    play('audio_raw.wav')    

def listen_music(keyword):
# keyword = input("请输入想要听的歌曲：")
    url = "http://songsearch.kugou.com/song_search_v2?callback=jQuery1124006980366032059648_1518578518932&keyword="+keyword+"&page=1&pagesize=30&userid=-1&clientver=&platform=WebFilter&tag=em&filter=2&iscorrection=1&privilege_filter=0&_=1518578518934"
    content = requests.get(url)
    if re.findall('"FileHash":"(.*?)"',content.text) == []:
        message_answer = '没有搜索到对应的歌曲'
        result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 0}) 
        with open('audio_answer.wav', 'wb') as f:
            f.write(result_tts)
        play('audio_answer.wav')
    else:
        filehash = re.findall('"FileHash":"(.*?)"',content.text)[0]
        songname = re.findall('"SongName":"(.*?)"',content.text)[0].replace("<\\/em>","").replace("<em>","") #即将播放的歌曲名
        hash_url = "http://www.kugou.com/yy/index.php?r=play/getdata&hash="+filehash
        hash_content = requests.get(hash_url)
        play_url = re.findall('"play_url":"(.*?)"',hash_content.text)
        play_url = ' '.join(play_url)
        real_download_url = play_url.replace("\\","")
        print("客官，请稍等一下，好音乐马上呈上！")
        # with open(songname+".mp3","wb")as fp:
        with open("music.mp3","wb")as fp:
            fp.write(requests.get(real_download_url).content)
        play('music.mp3')
        print('下载完成，敬请收听！')
    
    
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY) 
filename_record = 'AudioBase_DIY/ask_music1.wav'
# data = my_record()
# save_wave_file(filename_record,data)
# message = result_weather.encode('utf8')
# wf=wave.open(filename_record,'rb')
# rate=wf.getframerate()    #照目前的情况是8000，如果有变化，再用上这个语句
result_stt = client.asr(get_file_content(filename_record), 'wav', 8000, {'dev_pid': '1537'}) #dev_pid=1537是纯中文普通话，1536支持简单英文
#对wav文件无法识别没有RIFF头的文件，不能直接传data
# wf.close()

if result_stt['err_msg'] != 'success.':
    print('没有识别到内容')
else:
    print(result_stt['result'])
    message = str(result_stt['result']).encode('utf8')
#     message = '魔镜魔镜今天天气怎么样'.encode('utf8')
    search_result = re.search(u'.*魔镜'.encode('utf8'), message)
    if search_result is not None: #接收到了唤醒语，开始工作
        flag = 1
        search_result = re.search(u'.*天气'.encode('utf8'), message)
        if search_result is not None:
            message_answer = get_weather(message)
            say_answer(message_answer)
        else: 
            pattern = re.compile(u'歌|音乐'.encode('utf8'))
            search_result = re.findall(pattern, message)
            if len(search_result) != 0: #search失败返回的是Nonetype，findall失败返回一个空list
                message_answer = '你想听什么' + search_result[0].decode('utf8')
                say_answer(message_answer)
                # data = my_record() #再接收一波语音
                # save_wave_file(filename_record,data)
                filename_record = 'AudioBase_DIY/music_name3.wav' #临时语句，仅供测试
                result_stt = client.asr(get_file_content(filename_record), 'wav', 8000, {'dev_pid': '1537'})
                keyword = result_stt['result'][0].replace('，','') #str类型即可
                print(keyword)
#                 keyword = '告白气球'
                listen_music(keyword)                
            else:
                flag = 0
    #         message_answer = '你在说什么我听不见'
#         if flag:
#             result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 3}) 
#             if not isinstance(result_tts, dict):
#                 with open('audio_raw.wav', 'wb') as f:
#                     f.write(result_tts)
#             play('audio_raw.wav')

    # 首先实现：读取A23_0.wav，能将识别内容显示在屏幕上（已完成）
    # 然后实现：将显示在屏幕上的识别内容转成音频
    # 接着给音频加上RIFF头
    # 最后播放音频

In [ ]:
# -*- coding: utf-8 -*-
'''
sqllite3 操作
用于记录播放列表
'''
__author__ = 'Louie.v Check.vv@gmail.com)'

import sqlite3


dbName='./plist.db'

class DbSqlite():
    def __init__(self):
            '''
            创建数据库，如果库存在，不创建。
            '''
            creat_db_plist_sql='''
                CREATE TABLE if not exists "plist" (
                "id"  INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                "songid"  INTEGER NOT NULL,
                "songname"  TEXT(20) NOT NULL,
                "mp3url"  TEXT(200) NOT NULL,
                "artist"  TEXT(40),
                "album_name"  TEXT(40),
                "album_picurl"  TEXT(200),
                "quality"  TEXT(20),
                CONSTRAINT "listid" UNIQUE ("id" ASC),
                CONSTRAINT "songid" UNIQUE ("songid" ASC)
                );
                '''
            creat_db_seting_sql='''
            CREATE TABLE if not exists "seting" (
                "id"  INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                "name"  TEXT(20) NOT NULL,
                "value"  TEXT(100) NOT NULL,
                CONSTRAINT "id" UNIQUE ("id" ASC)
                CONSTRAINT "name" UNIQUE ("name" ASC)
                );
            '''


            try:
                conn=sqlite3.connect(dbName)
                cr=conn.cursor()
                cr.execute(creat_db_plist_sql)
                conn.commit()
                cr.execute(creat_db_seting_sql)
                conn.commit()
                fe=cr.fetchall()
                cr.close()
                conn.close()
            except sqlite3.Error as e:
                print ("On Create Table Error: ",e)

            # 管理密码初始化
            set_default_pwd_sql='''
            INSERT INTO "seting" ( "name", "value") VALUES ( 'pwd', 123456);
            '''
            pwd=self.select_seting_db("pwd")
            if not pwd:
                try:
                    conn=sqlite3.connect(dbName)
                    cr=conn.cursor()
                    cr.execute(set_default_pwd_sql)
                    conn.commit()
                    fe=cr.fetchall()
                    cr.close()
                    conn.close()
                except sqlite3.Error as e:
                    print ("On Set Default Pwd: ",e)
    def select_seting_db(self,name):
            '''
            设置查询
            '''
            conn=sqlite3.connect(dbName)
            cr=conn.cursor()
            cr.execute("select value from seting where name=\'" + name +"\';")
            fe=cr.fetchall()
            cr.close()
            conn.close()
            return  fe

    def select_db(self,sid):
            '''
            执行查询
            '''
            conn=sqlite3.connect(dbName)
            cr=conn.cursor()
            cr.execute("select songid from plist where songid=" + str(sid) +";")
            fe=cr.fetchall()
            cr.close()
            conn.close()
            return  fe

    def count_db(self):
            '''
            执行查询
            '''
            conn=sqlite3.connect(dbName)
            cr=conn.cursor()
            cr.execute("select count(*) from plist;")
            fe=cr.fetchall()
            cr.close()
            conn.close()
            return  fe
    def insert_db(self,dataDic):
            '''
            插入数据
            '''
            sqlDate=(dataDic["song_id"],dataDic["song_name"],dataDic["mp3_url"],dataDic["artist"],dataDic["album_name"],dataDic["album_picurl"],dataDic['quality'])
            try:
                conn=sqlite3.connect(dbName)
                cr=conn.cursor()
                cr.execute('''insert into plist ("songid","songname","mp3url","artist","album_name","album_picurl","quality") values (?,?,?,?,?,?,?)''',sqlDate)
                conn.commit()
                fe=cr.fetchall()
                cr.close()
                conn.close()
                return True
            except sqlite3.Error as e:
                print ("On Insert Error: ",e)
                return "insert db error"

    def delete_db(self,sid):
            '''
            删除数据
            '''
            try:
                conn=sqlite3.connect(dbName)
                cr=conn.cursor()
                cr.execute("delete from plist where songid="+ sid +";")
                conn.commit()
                fe=cr.fetchall()
                cr.close()
                conn.close()
                return True
            except sqlite3.Error as e:
                print ("On Delete Error: ",e)
                return "delete db error"
    def delete_all(self):
        '''
        删除所有
        '''
        try:
            conn=sqlite3.connect(dbName)
            cr=conn.cursor()
            cr.execute("delete from plist;")
            conn.commit()
            fe=cr.fetchall()
            cr.execute('update sqlite_sequence set seq=1 where name="plist";')
            conn.commit()
            fe=cr.fetchall()
            cr.close()
            conn.close()
            return  True
        except sqlite3 as e:
            print ("On Delete All Error:",e)
            return False

    def select_all(self):
        '''
        查询所有结果
        '''
        conn=sqlite3.connect(dbName)
        cr=conn.cursor()
        cr.execute("select songid,songname,artist from plist;")
        fe=cr.fetchall()
        cr.close()
        conn.close()
        return  fe
        print ("====================================select start================================")
        for row in fe:
            print (row)
        print ("=====================================select end================================")
        return fe

    def change_pwd(self,nowpwd,newpwd):
        '''
        修改密码
        '''
        try:
            conn=sqlite3.connect(dbName)
            cr=conn.cursor()
            cr.execute("select VALUE from seting WHERE name='pwd';")
            fe=cr.fetchall()
            __nowpwd=fe[0][0]
            if nowpwd == __nowpwd:
                try:
                    cr.execute("UPDATE SETING SET VALUE='"+newpwd+"' where name='pwd';")
                    conn.commit()
                    fe=cr.fetchall()
                    return True
                except sqlite3 as e:
                    print (e)
                    return False
            else:
                return False
        except e:
            print (e)
            return False



if __name__ == "__main__":
    db=DbSqlite()
    db.insert_db({
        'song_id':32358694,
        'song_name':"Brave Shine",
        'mp3_url':"""http://m1.music.126.net/NEl-rP7FHqjScMRA9HchfQ==/7886796906996700.mp3""",
        'artist':"Aimer",
        'album_name':"Brave Shine",
        'album_picurl':"http://p4.music.126.net/EnZBXon5cZWLPHEn2cHMFQ==/2940094094533735.jpg",
        'quality': ''
    })
    db.select_all()
    print (db.select_db(str(32358694)))

In [ ]:
import requests
import json
# r = requests.get('https://api.github.com/events') #返回响应为200的话就是正常的
# r = requests.post('http://httpbin.org/post', data={'key': 'value'})
# r = requests.put('http://httpbin.org/put', data = {'key': 'value'})
# r = requests.delete('http://httpbin.org/delete')
# r = requests.head('http://httpbin.org/get')
# r = requests.options('http://httpbin.org/get')
# r.url
# payload = {'key1': 'value1', 'key2': 'value2'}
# payload = {'key1': 'value1', 'key2': ['value2', 'value3']}
# payload = (('key1', 'value1'), ('key1', 'value2'))
# files = {'file': open('E:\我的东西\新建文件夹\文件\课件\信工课件\电子系统综合设计\新建文本文档.txt', 'rb')}
# r = requests.post('http://httpbin.org/post', files=files)
# payload = {'some': 'data'}
# r = requests.post(url, data=json.dumps(payload))
# url = 'https://api.github.com/'
# headers = {'user-agent': 'my-app/0.0.1'}
# r = requests.get(url, headers=headers) 
# r.url
# url = 'http://example.com/some/cookie/setting/url'
url = 'http://httpbin.org/cookies'
cookies = dict(cookies_are='working')
r = requests.get(url, cookies=cookies)

# r.cookies['example_cookie_name']
# r.json()
r.json()

In [ ]:
# r.headers
import http.cookiejar #旧版cookielib
import urllib #旧版urllib
filename = 'cookie.txt'
cookie = http.cookiejar.MozillaCookieJar(filename)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')
cookie.save(ignore_discard=True, ignore_expires=True)


In [ ]:
# for item in cookie:
#     print ('Name = ',item.name)
#     print ('Value = ',item.value)
cookie = http.cookiejar.MozillaCookieJar()
cookie.load(filename, ignore_discard=True, ignore_expires=True)
# req = urllib.request.Request("http://www.baidu.com")
# handler = urllib.request.HTTPCookieProcessor(cookie)
# opener = urllib.request.build_opener(handler)
postdata = urllib.parse.urlencode({'xh':'201530081693','pwd':r'dwdrs4836cjdbd!'}).encode('utf8')
# response = opener.open(req)
# aa = response.read()
# with open('music.html','wb') as f:
#     f.write(aa)
loginUrl = 'http://www.scut.edu.cn/jw2005/'
result_login = opener.open(loginUrl, postdata)
gradeUrl = 'http://www.scut.edu.cn/jw2005/' 
result_search = opener.open(gradeUrl)
print(result_login.read())